In [11]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.applications.xception import Xception
import os
from pickle import dump, load

In [12]:
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
     		if index == integer:
         	    return word
	return None

In [13]:
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

In [14]:
def extract_features(filename, model):
        try:
            image = Image.open(filename)

        except:
            print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
        image = image.resize((299,299))
        image = np.array(image)
        # for images that has 4 channels, we convert them into 3 channels
        if image.shape[2] == 4: 
            image = image[..., :3]
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        return feature

In [15]:
tokenizer = load(open("tokenizer.p","rb"))
model = load_model('models/model_1.h5')
xception_model = Xception(include_top=False, pooling="avg")
max_length = 32

In [16]:
def test_image(file):
    photo = extract_features(file, xception_model)
    description = generate_desc(model, tokenizer, photo, max_length)
    print(description)

In [17]:
test_image("img1.jpg")

start man in red shirt is walking on the beach end


In [18]:
test_image("img2.jpg")

start man in black shirt is standing on the street end


In [19]:
test_image("img3.jpg")

start man in red shirt is standing on the beach end


In [20]:
test_image("img4.jpeg")

start young boy in red shirt is playing with red ball end
